<a href="https://colab.research.google.com/github/KevinJayne2023/Lora_with_Flux_ComfyUI/blob/main/Flux_Lora_ComfyUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_CUSTOM_NODES` after first run if `USE_GOOGLE_DRIVE` is checked.


### Install Dependencies

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES'] = INSTALL_CUSTOM_NODES
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull


if OPTIONS['INSTALL_CUSTOM_NODES']:
  %cd $WORKSPACE
  %cd custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt
  !git clone https://github.com/Fannovel16/comfyui_controlnet_aux
  !pip install -r comfyui_controlnet_aux/requirements.txt

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies





## Log into HuggingFace

In [2]:
from huggingface_hub import login
login(new_session=True)

### Download Resources

In [ ]:
# Download the FLUX model from Hugging Face and save it in the ./models/unet directory
# !wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf" -P "./models/unet"

# Stronger FLUX model
# !wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q6_K.gguf" -P "/content/ComfyUI/models/unet"

!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q8_0.gguf" -P "/content/ComfyUI/models/unet"

!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-F16.gguf" -P "./models/unet"

!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-F16.gguf" -P "/content/ComfyUI/custom_nodes/"

# Download the CLIP T5 model from Hugging Face and save it in ./models/clip
# !wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf" -P "./models/clip"

# T5 Encoder for Q6_k GGUF
# !wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q6_K.gguf" -P "/content/ComfyUI/models/clip"

# !wget -c "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors" -P "./models/clip"

!wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-f16.gguf" -P "/content/ComfyUI/models/clip"

# Download the safetensors file for the CLIP model
# !wget -c "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors" -P "./models/clip"

# Download the Flux Text Encoder
!wget -c "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors" -P "/content/ComfyUI/models/clip"

# Download the LARGER t5xxl encoder
!wget -c "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors" -P "/content/ComfyUI/models/clip"

# Download the SMALLER t5xxl encoder
# !wget -c "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors" -P "./models/clip"

# Download the VAE model from Hugging Face and save it in ./models/vae
!wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "/content/ComfyUI/models/vae"

!wget -c "https://huggingface.co/black-forest-labs/FLUX.1-dev/resolve/main/ae.safetensors" -P "/content/ComfyUI/models/vae"

# ✅ Optional: Download VAE if needed
!wget -c "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors" -O "/content/ComfyUI/models/vae/vae-ft-mse-ema-pruned.safetensors"

# Download the LoRa model from Civitai.com Replace `xxxxxxxx` with your actual token
!wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=CIVIT_TOKEN_HERE" -O "/content/ComfyUI/models/loras/FluxThouS40k.safetensors"

# 📥 Download a base checkpoint model (e.g., Realistic Vision v5.1)
# !wget -c "https://civitai.com/api/download/models/128713" -O "/content/ComfyUI/models/checkpoints/realisticVisionV51_v51VAE.safetensors"

# Download NSFW Lora
# !wget -c "https://civitai.com/api/download/models/1050165?type=Model&format=SafeTensor" -O "./models/loras/nsfw.safetensors"

# Download the ControlNet models
# If you want to use a specific ControlNet model, remove the hashtag (#) from the corresponding line.
# !wget -c "https://huggingface.co/XLabs-AI/flux-controlnet-depth-v3/resolve/main/flux-depth-controlnet-v3.safetensors" -P "./models/controlnet"
# !wget -c "https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-canny-controlnet-v3.safetensors" -P "./models/controlnet"
# !wget -c "https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-hed-controlnet-v3.safetensors" -P "./models/controlnet"


## Loras and Checkpoint

In [ ]:
# 📥 Download a base checkpoint model (e.g., Realistic Vision v5.1)
# !wget -c "https://civitai.com/api/download/models/128713" -O "/content/ComfyUI/models/checkpoints/realisticVisionV51_v51VAE.safetensors"

# Download NSFW Lora
# !wget -c "https://civitai.com/api/download/models/1050165?type=Model&format=SafeTensor" -O "/content/ComfyUI/models/loras/nsfw.safetensors"

!wget -c "https://huggingface.co/strangerzonehf/Flux-Super-Realism-LoRA/resolve/main/super-realism.safetensors" -P "/content/ComfyUI/models/loras"

!wget -c "https://civitai.com/api/download/models/1920523?type=Model&format=SafeTensor&size=pruned&fp=fp16" -O "/content/ComfyUI/models/checkpoints/epicRealismXL_v10.safetensors"

!wget -c "https://civitai.com/api/download/models/1026423?type=Model&format=SafeTensor" -P "/content/ComfyUI/models/loras"

!wget -c "https://huggingface.co/XLabs-AI/flux-RealismLora/resolve/main/lora.safetensors" -O "/content/ComfyUI/models/loras/xlabs_RealismLora.safetensors"

!wget -c "https://huggingface.co/SG161222/Realistic_Vision_V6.0_B1_noVAE/resolve/main/Realistic_Vision_V6.0_NV_B1.safetensors" -P "/content/ComfyUI/models/checkpoints"


In [ ]:
!wget -c "https://huggingface.co/strangerzonehf/Flux-Super-Realism-LoRA/resolve/main/super-realism.safetensors" -P "/content/ComfyUI/models/loras"

!wget -c "https://civitai.com/api/download/models/1920523?type=Model&format=SafeTensor&size=pruned&fp=fp16" -O "/content/ComfyUI/models/checkpoints/epicRealismXL_v10.safetensors"


## Clear Unused Memory

### Run ComfyUI

In [ ]:
import gc
import torch

# Garbage collect unused memory
gc.collect()

# Clear GPU cache if using PyTorch (optional, for CUDA)
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

print("✅ RAM and CUDA cache cleared.")

In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server